In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
import pandas as pd
import re

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_4012\1378756733.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
udemy_cat_tag_df = pd.read_csv("../data-dump/udemy_lecture_cat_tags.csv")
udemy_cat_tag_df

,id,isAccess,category,tags
0,5467400,True,자격증,Professional Scrum Master(PSM)
1,1294046,True,모바일 개발,Android 개발
2,3620656,True,웹 개발,JavaScript
3,3825092,True,기타,WordPress
4,3843498,True,자격증,Tableau
...,...,...,...,...
59550,4693788,True,보안/네트워크,Powershell
59551,352378,True,기타,Python
59552,5510828,True,기타,JavaScript
59553,3381284,True,보안/네트워크,사이버 보안


In [2]:
def apply_logic(html_file, json_file):
    # 이 곳에 로직을 적용하세요.
    pass

# 폴더 경로
folder_path = "./page/" # 실제 폴더 경로로 변경해주세요.

html_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.html')}
json_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.json')}


In [3]:
def open_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
def open_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return BeautifulSoup(contents, 'html.parser')

In [5]:
str2level = {
    "초급자" : "BEGINNER",
    "중급자" : "INTERMEDIATE",
    "전문가" : "EXPERT",
    "모든 수준" : "ALL",
}
currency2won = {
    "CAD" : 996.94,
    "USD" : 1339.35
}

In [4]:
json_list = open_json("./list/parted/list.json")

In [44]:
def get_lecture_dic(pk):
    html_file = os.path.join(folder_path, f"{pk}.html")
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    json_info = json_list.get(pk)
    html_soup = open_html(html_file)
    
    lecture = {}

    sections = json_description["curriculum_context"]["data"]["sections"]
    total_time = 0
    for section in sections:
        total_time += section["content_length"]

    what_you_learn = list(map(lambda x:x.text, html_soup.find_all(class_="objective--objective-item--AHk7S")))


    curriculum = {}
    curriculum["curriculum"] = []
    for section in sections:
        temp = {}
        temp["time"] = int(section["content_length"]//60)
        temp["item_count"] = section["lecture_count"]
        temp["title"] = section["title"]
        items = []
        for item in section["items"]:
            items.append({
                "title" : item["title"],
                "time" : item["content_summary"]
            })
        temp["items"] = items
        curriculum["curriculum"].append(temp)

    price_dic = json_description["price_text"]["data"]["pricing_result"]["price"]
    sale_price_dic = json_description["price_text"]["data"]["pricing_result"]["list_price"]
    # int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
    # int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000

    # lecture["lecture_id"] = 1
    lecture["site_lecture_id"] = f"u{json_info["id"]}"
    lecture["name"] = json_info["title"]
    lecture["image"] = json_info["image_750x422"]
    lecture["category_name"] = udemy_cat_tag_df[udemy_cat_tag_df["id"]==json_info["id"]]["category"].item()
    # 을 넣어놓으면 조인해서 id받아올수있다. 
    # lecture["category_id"]
    lecture["price_original"] = int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
    lecture["price_sale"] = int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000
    lecture["level"] = str2level[json_info["instructional_level_simple"]]
    lecture["summary"] = "||".join(list(map(lambda x:x, what_you_learn)))
    lecture["description_summary"] = json_info["headline"]
    # lecture["description_detail"] html이라 일단 더미
    lecture["site_review_rating"] = json_description["slider_menu"]["data"]["rating"]
    lecture["site_review_count"] = json_description["slider_menu"]["data"]["num_reviews"]
    lecture["site_student_count"] = json_description["slider_menu"]["data"]["num_students"]
    lecture["review_sum"] = 0
    lecture["review_count"] = 0
    lecture["total_time"] = int(total_time//60)
    lecture["curriculum"] = curriculum
    lecture["instructor"] = json_info["visible_instructors"][0]["title"]
    lecture["site_type"] = "UDEMY"
    lecture["site_link"] = f"https://udemy.com{json_info["url"]}"
    # lecture["gpt_review_good"] = 
    # lecture["gpt_review_bad"] = 
    # lecture["create_time"] = 
    # lecture["update_time"] = 

    return lecture

In [5]:
udemy_lectures = []
times = set()
for pk in tqdm((html_files & json_files)):
    try:
        json_file = os.path.join(folder_path, f"{pk}.json")
        json_description = open_json(json_file)
        json_info = json_list.get(pk)
        
        curriculum = {}
        curriculum["curriculum"] = []
        sections = json_description["curriculum_context"]["data"]["sections"]

        for section in sections:
            for item in section["items"]:
                times.add(item["content_summary"])
    except:
        continue


  0%|          | 0/60130 [00:00<?, ?it/s]

 14%|█▎        | 8205/60130 [00:20<02:00, 431.77it/s]

In [9]:
times

{'',
 '01:52:05',
 '02:50',
 '15:55',
 '46:53',
 '1 trang',
 '11 întrebări',
 '02:50:37',
 '01:25:35',
 '01:05:11',
 '01:28:10',
 '200 questions',
 '31 pertanyaan',
 '06:43',
 '48:40',
 '25 вопросов',
 '41:17',
 '39:25',
 '02:00:56',
 '55:55',
 '01:58:16',
 '9 问题',
 '01:34',
 '32:58',
 '01:02:48',
 '1 domande',
 '00:26',
 '17:15',
 '43:48',
 '01:33',
 '01:18:47',
 '01:08:02',
 '01:49:00',
 '56:40',
 '01:38:08',
 '01:27:20',
 '01:30:27',
 '02:36:27',
 '01:31:22',
 '04:55',
 '01:19:29',
 '04:03:25',
 '7.4 MB',
 '13 soru',
 '02:36:41',
 '02:57:33',
 '33:12',
 '01:06:47',
 '02:06:14',
 '02:06:54',
 '08:35',
 '31:57',
 '34:21',
 '09:46',
 '31:35',
 '02:15:18',
 '46:35',
 '55:22',
 '13 questions',
 '06:01',
 '47:47',
 '01:51',
 '03:48',
 '01:52:18',
 '01:42:18',
 '17:35',
 '45:21',
 '51:34',
 '51:58',
 '41:14',
 '02:18:37',
 '10:58',
 '36:39',
 '53:01',
 '02:09:58',
 '13:01',
 '54:11',
 '01:56:17',
 '31:18',
 '02:46',
 '42:52',
 '01:01:50',
 '27:55',
 '01:06:11',
 '21:47',
 '02:25',
 '02:40:

In [51]:
udemy_lectures = []
page = None
for pk in tqdm((html_files & json_files)):
    try:
        udemy_lectures.append(get_lecture_dic(pk))
    except Exception as e:
        continue

  8%|▊         | 5038/60130 [08:07<1:28:49, 10.34it/s]


KeyboardInterrupt: 

In [50]:
udemy_cat_tag_df[udemy_cat_tag_df["id"]==page]

,id,isAccess,category,tags


In [49]:
print(udemy_cat_tag_df[udemy_cat_tag_df["id"]==page]["category"].item())
        

ValueError: can only convert an array of size 1 to a Python scalar

In [43]:
what_you_learn = list(map(lambda x:x.text, page.find_all(class_="objective--objective-item--AHk7S")))
"||".join(list(map(lambda x:x, what_you_learn)))


'How to Install the Steam Advanced Sessions plugin.||How to host an online server on Steam.||How to design & create a fully functional Server Browser.||How to filter for servers through server names, languages & more.||How to create, search and join online game servers.||How to package the project for friends and family to play together.'

In [33]:

list(map(lambda x:x.text, page.find_all(class_="objective--objective-item--AHk7S")))


['How to Install the Steam Advanced Sessions plugin.',
 'How to host an online server on Steam.',
 'How to design & create a fully functional Server Browser.',
 'How to filter for servers through server names, languages & more.',
 'How to create, search and join online game servers.',
 'How to package the project for friends and family to play together.']

In [20]:
pd.DataFrame(udemy_lectures)

,site_lecture_id,name,image,category_name,price_original,price_sale,level,summary,description_summary,site_review_rating,site_review_count,site_student_count,review_sum,review_count,total_time,curriculum,instructor,site_type,site_link
0,u4429452,Visual Basic - VB dot Net Programming,https://img-c.udemycdn.com/course/750x422/4429...,프로그래밍 언어,15000,30000,ALL,The Introduction to VB .Net||Control Propertie...,Learn Visual Basic dot Net and Windows Forms b...,4.204759,73,5477,0,0,66,"{'curriculum': [{'time': 66, 'item_count': 19,...",DIGIFLAX E-LEARNING,UDEMY,https://udemy.com/course/vb-dot-net-programming/


In [8]:
price_dic = json_description["price_text"]["data"]["pricing_result"]["price"]
sale_price_dic = json_description["price_text"]["data"]["pricing_result"]["list_price"]
int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000

30000

In [ ]:
udemy_cat_tag_df[udemy_cat_tag_df["id"]==352378]["tags"].item()
udemy_cat_tag_df[udemy_cat_tag_df["id"]==352378]["category"].item()

In [5]:
for pk in tqdm((html_files & json_files)):
    html_file = os.path.join(folder_path, f"{pk}.html")
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    json_info = json_list.get(pk)
    html_soup = open_html(html_file)
    
    lecture = {}

    sections = json_description["curriculum_context"]["data"]["sections"]
    total_time = 0
    for section in sections:
        total_time += section["content_length"]

    curriculum = {}
    curriculum["curriculum"] = []
    for section in sections:
        temp = {}
        temp["time"] = int(section["content_length"]//60)
        temp["item_count"] = section["lecture_count"]
        temp["title"] = section["title"]
        items = []
        for item in section["items"]:
            items.append({
                "title" : item["title"],
                "time" : item["content_summary"]
            })
        temp["items"] = items
        curriculum["curriculum"].append(temp)
        json.dumps(curriculum)


    print(curriculum)

    break

  0%|          | 0/60130 [00:00<?, ?it/s]

  0%|          | 0/60130 [00:00<?, ?it/s]

{'curriculum': [{'time': 28, 'item_count': 6, 'title': 'Introduction to source code management and version control', 'items': [{'title': 'Intro', 'time': '00:18'}, {'title': 'Welcome to course', 'time': '01:02'}, {'title': 'What you will learn', 'time': '06:09'}, {'title': 'Introduction', 'time': '01:06'}, {'title': 'Understanding source code management', 'time': '09:18'}, {'title': 'What’s version control', 'time': '10:18'}]}, {'time': 2, 'item_count': 1, 'title': 'GIT history, installation and tools', 'items': [{'title': 'Understanding GIT', 'time': '02:33'}]}, {'time': 21, 'item_count': 3, 'title': 'Understanding and working with GIT', 'items': [{'title': 'How to work with GIT - P1', 'time': '07:38'}, {'title': 'How to work with GIT - P2', 'time': '10:02'}, {'title': 'How to work with GIT - P3', 'time': '03:58'}]}, {'time': 12, 'item_count': 1, 'title': 'Basic GIT Commands', 'items': [{'title': 'GIT CMD', 'time': '12:19'}]}, {'time': 11, 'item_count': 2, 'title': 'Git Diff and Merge

In [6]:
json.dumps(curriculum)

'{"curriculum": [{"time": 28, "item_count": 6, "title": "Introduction to source code management and version control", "items": [{"title": "Intro", "time": "00:18"}, {"title": "Welcome to course", "time": "01:02"}, {"title": "What you will learn", "time": "06:09"}, {"title": "Introduction", "time": "01:06"}, {"title": "Understanding source code management", "time": "09:18"}, {"title": "What\\u2019s version control", "time": "10:18"}]}, {"time": 2, "item_count": 1, "title": "GIT history, installation and tools", "items": [{"title": "Understanding GIT", "time": "02:33"}]}, {"time": 21, "item_count": 3, "title": "Understanding and working with GIT", "items": [{"title": "How to work with GIT - P1", "time": "07:38"}, {"title": "How to work with GIT - P2", "time": "10:02"}, {"title": "How to work with GIT - P3", "time": "03:58"}]}, {"time": 12, "item_count": 1, "title": "Basic GIT Commands", "items": [{"title": "GIT CMD", "time": "12:19"}]}, {"time": 11, "item_count": 2, "title": "Git Diff an

# re curriculum

In [34]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
import pandas as pd
import re


def getTime():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def send_alert_to_mattermost(message):
    webhook_url = "https://meeting.ssafy.com/hooks/ro9xkennxbrhpdh639re3w9jha"
     # 여기에 Mattermost에서 얻은 Webhook URL을 입력해주세요

    payload = {
        "text": message
    }

    response = requests.post(webhook_url, json=payload)
    if response.status_code == 200:
        logging.info(f'{getTime()} Alert가 성공적으로 전송되었습니다.')
    else:
        logging.info(f'{getTime()} Alert가 실패했습니다.')



# 폴더 경로
folder_path = "./page/" # 실제 폴더 경로로 변경해주세요.

html_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.html')}
json_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.json')}

def open_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
def open_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return BeautifulSoup(contents, 'html.parser')

json_list = open_json("./list/parted/list.json")

def get_lecture_dic(pk):
    html_file = os.path.join(folder_path, f"{pk}.html")
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    
    lecture = {}

    sections = json_description["curriculum_context"]["data"]["sections"]
    total_time = 0
    for section in sections:
        total_time += section["content_length"]

    curriculum = {}
    curriculum["curriculum"] = []
    for section in sections:
        temp = {}
        temp["time"] = int(section["content_length"]//60)
        temp["item_count"] = section["lecture_count"]
        temp["title"] = section["title"]
        items = []
        for item in section["items"]:
            items.append({
                "title" : item["title"],
                "time" : item["content_summary"]
            })
        temp["items"] = items
        curriculum["curriculum"].append(temp)
        
    lecture["pk"] = "u"+pk
    lecture["original_curriculum"] = curriculum
    lecture["dumped_curriculum"] = json.dumps(curriculum)


    return lecture

udemy_lectures = []
send_alert_to_mattermost("udemy lectures 시작")
for pk in tqdm((html_files & json_files)):
    try:
        udemy_lectures.append(get_lecture_dic(pk))
    except Exception as e:
        continue

pd.DataFrame(udemy_lectures).to_csv("./new_udemy_lectures_dummy_curriculum.csv",
                                    index=False)
send_alert_to_mattermost("udemy lectures 끝")


100%|██████████| 60130/60130 [01:40<00:00, 599.46it/s] 


# price update

In [4]:
html_files

{'4742876',
 '3855138',
 '3517910',
 '361980',
 '4791448',
 '5326654',
 '1686452',
 '4698450',
 '5171394',
 '5560062',
 '4117256',
 '4378896',
 '5187600',
 '5560982',
 '2669586',
 '5195520',
 '2971498',
 '5663854',
 '1839318',
 '5722042',
 '5185716',
 '1669038',
 '5234724',
 '3033028',
 '4481016',
 '4604912',
 '5255778',
 '3917042',
 '4929436',
 '4424250',
 '4915016',
 '4493808',
 '2517524',
 '5509740',
 '5202864',
 '5476800',
 '883032',
 '4631578',
 '4515100',
 '376460',
 '5258956',
 '3887266',
 '4143016',
 '5304338',
 '4952352',
 '5487324',
 '5513000',
 '5664094',
 '4078114',
 '3934790',
 '2591174',
 '5702820',
 '4224666',
 '2080882',
 '3856192',
 '4305931',
 '4712236',
 '5666290',
 '3866668',
 '4140724',
 '5642440',
 '2433464',
 '3534090',
 '5391836',
 '5573662',
 '2876082',
 '3377854',
 '2507588',
 '2037118',
 '3833824',
 '1321534',
 '4009944',
 '3898480',
 '4300701',
 '5178688',
 '4125660',
 '4104714',
 '2131354',
 '5584678',
 '5356882',
 '4357708',
 '4048184',
 '2591686',
 '29754

In [24]:
price_root = "./price/"
price_dic = dict()
sum = 0
for i in range(0,10):
    folder_path = f"./price/price_{i}/page/"
    json_files = {f for f in os.listdir(folder_path) if f.endswith('.json')}
    for json_file in tqdm(json_files):
        pk = json_file[:-5].split("_")[1]
        data = open_json(folder_path+json_file)
        if ("price_text" not in data):
            sum+=1
            continue
        prices = data["price_text"]["data"]["pricing_result"]
        if (prices["price"]["currency"]!="KRW"):
            sum+=1
            continue
        if (pk in price_dic):
            continue

        price_dic[pk] = prices

        # print(data)
        # break
        # pk = json_file.s
    # print(json_files)
    # break

  0%|          | 0/10456 [00:00<?, ?it/s]

100%|██████████| 6221/6221 [00:03<00:00, 1854.84it/s]


In [29]:
price_list = []
for pk, item in price_dic.items():
    temp = {}
    price_sale = int(item["price"]["amount"])
    price_original = int(item["list_price"]["amount"])
    temp["site_lecture_id"] = "u"+pk
    temp["price_original"] = price_original
    temp["price_sale"] = price_sale
    price_list.append(temp)
    # break
price_df = pd.DataFrame(price_list)
price_df

,site_lecture_id,price_original,price_sale
0,u1234936,25000,25000
1,u19387,25000,17000
2,u1193516,25000,25000
3,u1519414,25000,25000
4,u583238,25000,17000
...,...,...,...
62280,u5619340,49000,49000
62281,u5672208,49000,49000
62282,u5622892,59000,59000
62283,u5697712,59000,59000


In [35]:
curriculum_df = pd.read_csv("./new_udemy_lectures_dummy_curriculum.csv")
curriculum_df

,pk,original_curriculum,dumped_curriculum
0,u4742876,"{'curriculum': [{'time': 3, 'item_count': 4, '...","{""curriculum"": [{""time"": 3, ""item_count"": 4, ""..."
1,u3855138,"{'curriculum': [{'time': 0, 'item_count': 0, '...","{""curriculum"": [{""time"": 0, ""item_count"": 0, ""..."
2,u3517910,"{'curriculum': [{'time': 1, 'item_count': 1, '...","{""curriculum"": [{""time"": 1, ""item_count"": 1, ""..."
3,u361980,"{'curriculum': [{'time': 0, 'item_count': 0, '...","{""curriculum"": [{""time"": 0, ""item_count"": 0, ""..."
4,u4791448,"{'curriculum': [{'time': 60, 'item_count': 8, ...","{""curriculum"": [{""time"": 60, ""item_count"": 8, ..."
...,...,...,...
60107,u1419430,"{'curriculum': [{'time': 144, 'item_count': 16...","{""curriculum"": [{""time"": 144, ""item_count"": 16..."
60108,u3545512,"{'curriculum': [{'time': 77, 'item_count': 5, ...","{""curriculum"": [{""time"": 77, ""item_count"": 5, ..."
60109,u5447714,"{'curriculum': [{'time': 499, 'item_count': 28...","{""curriculum"": [{""time"": 499, ""item_count"": 28..."
60110,u5265908,"{'curriculum': [{'time': 5, 'item_count': 2, '...","{""curriculum"": [{""time"": 5, ""item_count"": 2, ""..."


In [37]:
merged_df = pd.merge(curriculum_df, price_df, left_on='pk', right_on='site_lecture_id', how='inner')
merged_df = merged_df.drop(columns=["pk"])
merged_df

,original_curriculum,dumped_curriculum,site_lecture_id,price_original,price_sale
0,"{'curriculum': [{'time': 3, 'item_count': 4, '...","{""curriculum"": [{""time"": 3, ""item_count"": 4, ""...",u4742876,23000,23000
1,"{'curriculum': [{'time': 0, 'item_count': 0, '...","{""curriculum"": [{""time"": 0, ""item_count"": 0, ""...",u3855138,23000,23000
2,"{'curriculum': [{'time': 1, 'item_count': 1, '...","{""curriculum"": [{""time"": 1, ""item_count"": 1, ""...",u3517910,25000,25000
3,"{'curriculum': [{'time': 0, 'item_count': 0, '...","{""curriculum"": [{""time"": 0, ""item_count"": 0, ""...",u361980,25000,17000
4,"{'curriculum': [{'time': 60, 'item_count': 8, ...","{""curriculum"": [{""time"": 60, ""item_count"": 8, ...",u4791448,23000,23000
...,...,...,...,...,...
60106,"{'curriculum': [{'time': 144, 'item_count': 16...","{""curriculum"": [{""time"": 144, ""item_count"": 16...",u1419430,25000,25000
60107,"{'curriculum': [{'time': 77, 'item_count': 5, ...","{""curriculum"": [{""time"": 77, ""item_count"": 5, ...",u3545512,25000,25000
60108,"{'curriculum': [{'time': 499, 'item_count': 28...","{""curriculum"": [{""time"": 499, ""item_count"": 28...",u5447714,49000,49000
60109,"{'curriculum': [{'time': 5, 'item_count': 2, '...","{""curriculum"": [{""time"": 5, ""item_count"": 2, ""...",u5265908,0,0


In [38]:
merged_df.to_csv("./new_udemy_lectures_dummy_curriculum.csv",index=False)

In [ ]:
lectures_df = pd.read_csv("./lectures/udemy_lectures_dummy.csv")

In [41]:
lectures_df = pd.read_csv("./lectures/udemy_lectures_dummy.csv")
lectures_df = lectures_df.drop(columns=["price_original","price_sale","curriculum"])
merged_lecture_df = pd.merge(lectures_df, merged_df, how="inner", left_on="site_lecture_id", right_on="site_lecture_id")
merged_lecture_df = merged_lecture_df.drop(columns=["original_curriculum"])
merged_lecture_df.rename(columns = {'dumped_curriculum' : 'curriculum'}, inplace = True)
merged_lecture_df.to_csv("./lectures/new_udemy_lectures_dummy.csv",index=False)


In [39]:
lectures_df = pd.read_csv("./lectures/udemy_lectures_dummy.csv")

,site_lecture_id,name,image,category_name,price_original,price_sale,level,summary,description_summary,site_student_count,review_sum,review_count,total_time,curriculum,instructor,site_type,site_link,site_review_rating,site_review_count
0,u4452012,Powershell : Create a desktop application GUI,https://img-c.udemycdn.com/course/750x422/4452...,기타,100000,20000,INTERMEDIATE,Create a desktop application using powershell|...,Develop a standalone application Tool using Po...,2206,0,0,241,"{'curriculum': [{'time': 4, 'item_count': 1, '...",Vishnu Gupthan Nair,UDEMY,https://udemy.com/course/powershell-create-a-d...,4.346154,26
1,u3429610,Complete Core Java for Testers+Interview Programs,https://img-c.udemycdn.com/course/750x422/3429...,프로그래밍 언어,25000,15000,ALL,You will get comprehensive knowledge of Java P...,Best course on Java Programming for complete b...,70,0,0,488,"{'curriculum': [{'time': 5, 'item_count': 1, '...",Tech Planet,UDEMY,https://udemy.com/course/complete-core-java-fo...,4.309524,21
2,u5168890,CompTIA A+ Core 2 (220-1102) Exam Questions an...,https://img-c.udemycdn.com/course/750x422/5168...,자격증,25000,15000,ALL,Exam Domain 1.0 Operating Systems - 31%||Exam ...,Pass A+ Certification Core 2 (220-1102) Exam G...,2,0,0,0,"{'curriculum': [{'time': 0, 'item_count': 0, '...",Birali Faustin Nzubahimana,UDEMY,https://udemy.com/course/comptia-a-core-2-220-...,0.000000,0
3,u2119890,الدليل الشامل لتعلم بايثون 2022,https://img-c.udemycdn.com/course/750x422/2119...,프로그래밍 언어,25000,15000,ALL,هنعرف يعني ايه برمجه||هنتعرف علي تاريخ لغه برم...,الدليل الشامل لتعلم البرمجه ببايثون من الصفر ب...,25258,0,0,537,"{'curriculum': [{'time': 2, 'item_count': 4, '...",Mahmoud Ahmed,UDEMY,https://udemy.com/course/learn-python-arabic/,4.402680,1567
4,u1740734,HAProxy for Beginners,https://img-c.udemycdn.com/course/750x422/1740...,기타,25000,25000,ALL,Students will be able to build a high availabl...,Load Balancing and High Availability,2964,0,0,102,"{'curriculum': [{'time': 102, 'item_count': 16...",Hadi Alnabriss,UDEMY,https://udemy.com/course/haproxy-a/,4.173212,713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59530,u5129420,취업 준비생을 위한 Git (소스코드 버전관리),https://img-c.udemycdn.com/course/750x422/5129...,웹 개발,25000,15000,BEGINNER,소스 코드 형상 관리 기본 이해||macOS / Windows 터미널 명령어 이해|...,"[2023 취업대비] 디자인, 퍼블리싱 직군 취업 준비생을 위해 실습을 구성하였습니다.",173,0,0,41,"{'curriculum': [{'time': 7, 'item_count': 2, '...",황선수닷컴 Hwang Sunsoo,UDEMY,https://udemy.com/course/command-line-git-basic/,4.593750,80
59531,u4083098,Create a Smart Contract on WAX blockchain incl...,https://img-c.udemycdn.com/course/750x422/4083...,프로그래밍 언어,70000,20000,INTERMEDIATE,How to setup a environment in Docker to develo...,Learn how to interact with blockchain Oracles ...,618,0,0,306,"{'curriculum': [{'time': 3, 'item_count': 1, '...",Icaro Lavrador,UDEMY,https://udemy.com/course/wax-blockchain/,4.038095,105
59532,u1181318,Learn 2D Character Animation for Games with Bl...,https://img-c.udemycdn.com/course/750x422/1181...,게임 개발,70000,20000,INTERMEDIATE,Learn how to model and texture map 2D Characte...,"Explore how to build, animate, and setup 2D ga...",521,0,0,296,"{'curriculum': [{'time': 60, 'item_count': 12,...",3dmotive LLC,UDEMY,https://udemy.com/course/3dmotive-learn-2d-cha...,4.229730,37
59533,u3772896,"Weblogic Performance Tuning, Patching & Upgrad...",https://img-c.udemycdn.com/course/750x422/3772...,자격증,65000,15000,ALL,Fundamentals of performance tuning||Basic Term...,The Complete Industrial Approach For Certifica...,238,0,0,201,"{'curriculum': [{'time': 31, 'item_count': 5, ...",Sandip Jha,UDEMY,https://udemy.com/course/weblogic-performance-...,3.982759,58


# description summary

In [46]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
import pandas as pd
import re


udemy_cat_tag_df = pd.read_csv("../data-dump/udemy_lecture_cat_tags.csv")

# 폴더 경로
folder_path = "./page/" # 실제 폴더 경로로 변경해주세요.

html_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.html')}
json_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.json')}

def open_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
def open_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return BeautifulSoup(contents, 'html.parser')

json_list = open_json("./list/parted/list.json")

udemy_lectures = []
for pk in tqdm((html_files & json_files)):
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    json_info = json_list.get(pk)
    
    lecture = {}

    lecture["site_lecture_id"] = f"u{pk}"

    if("headline" not in json_info):
        continue
    lecture["description_summary"] = json_info["headline"]
    udemy_lectures.append(lecture)


pd.DataFrame(udemy_lectures).to_csv("./udemy_lectures_dummy.csv",
                                    index=False)


  0%|          | 120/60130 [00:10<1:28:01, 11.36it/s]


KeyboardInterrupt: 

In [51]:
pd.DataFrame(udemy_lectures)["description_summary"].isna().value_counts()

description_summary
False    120
Name: count, dtype: int64

In [45]:
json_info["headline"]

'Apprenez à utiliser Bootstrap 5 pour créer des pages Web dynamiques'